In [259]:
import bentoml
from bentoml.io import JSON, Text
import pandas as pd
import random
import numpy as np
import requests
import json

In [260]:
# Load input CSV file to analyze its content
file_path = 'failure_data/health_index_augdata.csv'
health_index_data = pd.read_csv(file_path)

In [261]:
print(health_index_data.iloc[14])

Hydrogen                                                            9040
Oxigen                                                              1870
Nitrogen                                                           47500
Methane                                                             1790
CO                                                                   183
CO2                                                                 2060
Ethylene                                                            7820
Ethane                                                               638
Acethylene                                                          1450
DBDS                                                                 0.0
Power factor                                                         1.0
Interfacial V                                                         42
Dielectric rigidity                                                   55
Water content                                      

In [262]:
json_string = health_index_data.iloc[14].to_json()
print(json_string)

{"Hydrogen":9040,"Oxigen":1870,"Nitrogen":47500,"Methane":1790,"CO":183,"CO2":2060,"Ethylene":7820,"Ethane":638,"Acethylene":1450,"DBDS":0.0,"Power factor":1.0,"Interfacial V":42,"Dielectric rigidity":55,"Water content":18,"Health index":60.5,"Life expectation":19.0,"CO_H2_ratio":0.0202433628,"CH4_H2_ratio":0.1980088496,"C2H4_H2_ratio":0.8650442478,"C2H2_H2_ratio":0.1603982301,"H2_N2_ratio":0.1903157895,"O2_N2_ratio":0.0393684211,"H2_CO2_ratio":4.3883495146,"TransformerID":"TX-8395","InstallationDate":"2004-10-09","MaintenanceSchedule":"2023-04-20","ReplacementHistory":"No major replacements","MaintenanceID":"MNT-61000","MaintenanceType":"Corrective Maintenance","MaintenanceDate":"2023-12-21","TransformerLoad":223.9613072983,"Temperature":52.7606643498,"Vibration":4.9950850466,"Humidity":41.0231441246,"Precipitation":37.1465164135,"Alerts":null,"IncidentType":null,"IncidentDetails":"No incidents","OperationID":"OP-7963","OperatingTime":34.7907773926,"LoadCondition":"Normal Load","Perfo

In [263]:
 # Convert input JSON data into a pandas DataFrame (for compatibility)
input_df_test = json.loads(json_string)
# create a pandas DataFrame from the dictionary
input_data = pd.DataFrame([input_df_test])
# print the DataFrame
print(input_data)

   Hydrogen  Oxigen  Nitrogen  Methane   CO   CO2  Ethylene  Ethane  \
0      9040    1870     47500     1790  183  2060      7820     638   

   Acethylene  DBDS  ...  Vibration   Humidity  Precipitation  Alerts  \
0        1450   0.0  ...   4.995085  41.023144      37.146516    None   

   IncidentType  IncidentDetails  OperationID  OperatingTime  LoadCondition  \
0          None     No incidents      OP-7963      34.790777    Normal Load   

                                  PerformanceMetrics  
0  Performance stable with a load of 223.96130729...  

[1 rows x 42 columns]


In [264]:
# Generate a text-based report using only input data
report = f"""
    Transformer ID: {input_data['TransformerID'].iloc[0]}
    InstallationDate: {input_data['InstallationDate'].iloc[0]}
    OperatingTime: {input_data['OperatingTime'].iloc[0]} years
    LoadCondition: {input_data['LoadCondition'].iloc[0]}
    PerformanceMetrics: {input_data['PerformanceMetrics'].iloc[0]}

    Condition Summary:
    Health Index: {input_data['Health index'].iloc[0]}
    Predicted Remaining Life Expectancy: {input_data['Life expectation'].iloc[0]} years
    Based on the current dissolved gas analysis (DGA) and health index data, this transformer is expected to last an additional {input_data['Life expectation'].iloc[0]} years. 

    Specific gases dissolved in the transformer insulating oil, could help identify potential faults or degradation. Current levels are:
    - Hydrogen: {input_data['Hydrogen'].iloc[0]} ppm.
    - Oxygen: {input_data['Oxigen'].iloc[0]} ppm.
    - Nitrogen: {input_data['Nitrogen'].iloc[0]} ppm.
    - Methane: {input_data['Methane'].iloc[0]} ppm.
    - CO: {input_data['CO'].iloc[0]} ppm.
    - CO2: {input_data['CO2'].iloc[0]} ppm.
    - Ethylene: {input_data['Ethylene'].iloc[0]} ppm.
    - Ethane: {input_data['Ethane'].iloc[0]} ppm.
    - Acethylene: {input_data['Acethylene'].iloc[0]} ppm.
    - DBDS: {input_data['DBDS'].iloc[0]} ppm.

    Other Key indicators suggest that:
    - Power factor: {input_data['Power factor'].iloc[0]}.
    - Interfacial V: {input_data['Interfacial V'].iloc[0]}.
    - Dielectric rigidity: {input_data['Dielectric rigidity'].iloc[0]}.
    - Water content: {input_data['Water content'].iloc[0]} ppm.

    Calculated Gas ratios are:
    - Carbon Monoxide to Hydrogen Ratio: {input_data['CO_H2_ratio'].iloc[0]}.
    - Methane to Hydrogen Ratio: {input_data['CH4_H2_ratio'].iloc[0]}.
    - Ethylene to Hydrogen Ratio: {input_data['C2H4_H2_ratio'].iloc[0]}.
    - Acetylene to Hydrogen Ratio: {input_data['C2H2_H2_ratio'].iloc[0]}.

    Maintenance Information:
    - Maintenance ID: {input_data['MaintenanceID'].iloc[0]}
    - Maintenance Type: {input_data['MaintenanceType'].iloc[0]} 
    - Last Maintenance Date: {input_data.get('MaintenanceDate', 'N/A')[0]}
    - ReplacementHistory: {input_data.get('ReplacementHistory', 'N/A')[0]}

    Environmental and Operational Factors:
    - Transformer Load (KW): {input_data.get('TransformerLoad', 'N/A')[0]}
    - Temperature (Celsius): {input_data.get('Temperature', 'N/A')[0]}
    - Humidity (%): {input_data.get('Humidity', 'N/A')[0]}
    - Vibration Level: {input_data.get('Vibration', 'N/A')[0]} cycles.
    - Precipitation Level: {input_data.get('Precipitation', 'N/A')[0]} mm.

    Alerts & Incidents:
    - Alerts: {input_data.get('Alerts', 'None')[0]}.
    - Incident Type: {input_data.get('IncidentType', 'None')[0]}.
    - Incident Details: {input_data.get('IncidentDetails', 'None')[0]}.

    Create Best Plan for Maintenance and Risk reduction Plan Recommendation:
"""

print(report)


    Transformer ID: TX-8395
    InstallationDate: 2004-10-09
    OperatingTime: 34.7907773926
    LoadCondition: Normal Load
    PerformanceMetrics: Performance stable with a load of 223.96130729828337 KW and temperature 52.76066434980563°C

    Condition Summary:
    Health Index: 60.5
    Predicted Remaining Life Expectancy: 19.0 years
    Based on the current dissolved gas analysis (DGA) and health index data, this transformer is 
    expected to last an additional 19.0 years. 

    Specific gases dissolved in the transformer insulating oil, could help identify potential faults or degradation. Current levels are:
    - Hydrogen: 9040
    - Oxygen: 1870
    - Nitrogen: 47500
    - Methane: 1790
    - CO: 183
    - CO2: 2060
    - Ethylene: 7820
    - Ethane: 638
    - Acethylene: 1450
    - DBDS: 0.0

    Other Key indicators suggest that:
    - Power factor: 1.0
    - Interfacial V: 42
    - Dielectric rigidity: 55
    - Water content: 18

    Calculated Gas ratios are:
    - Carbo

## Sumarization Example with summarize_transformer_report.py service

In [265]:
with bentoml.SyncHTTPClient("http://localhost:3000") as client:
    result = client.summarize(
        json_string=health_index_data.iloc[14].to_json()
    )

formatted_result = result.replace(". ", ".\n")
print(formatted_result)

 The transformer is expected to last an additional 19.0 years .
The current dissolved gas analysis (DGA) and health index data, could help identify potential faults or degradation .
The transformer has been installed in 2004-10-09 and it is currently stable with a load of 223.96130729828337 KW and temperature 52.76066434980563°C .
There have been no major replacements and the transformer has not been replaced since 2010 .
It has been placed in service to replace the transformer with a new, improved transformer that is capable of carrying more than a million gallons of oil .
The power factor is 1.0% and the expected life expectancy of the transformer is 19.5% .
The predicted remaining life expectancy is 18.0%.
The transformer’s maintenance has been replaced with a “preventation and risk reduction plan” has been put in place for the transformer” for the most of the last time since 2004-12-11


In [267]:
report_filename = "output_data/" + input_data['TransformerID'].iloc[0] + "_report.txt"
print(report_filename)

output_data/TX-8395_report.txt


In [268]:
final_report = report + formatted_result
print(final_report)

with open(report_filename, "w") as f:
    f.write(final_report)


    Transformer ID: TX-8395
    InstallationDate: 2004-10-09
    OperatingTime: 34.7907773926
    LoadCondition: Normal Load
    PerformanceMetrics: Performance stable with a load of 223.96130729828337 KW and temperature 52.76066434980563°C

    Condition Summary:
    Health Index: 60.5
    Predicted Remaining Life Expectancy: 19.0 years
    Based on the current dissolved gas analysis (DGA) and health index data, this transformer is 
    expected to last an additional 19.0 years. 

    Specific gases dissolved in the transformer insulating oil, could help identify potential faults or degradation. Current levels are:
    - Hydrogen: 9040
    - Oxygen: 1870
    - Nitrogen: 47500
    - Methane: 1790
    - CO: 183
    - CO2: 2060
    - Ethylene: 7820
    - Ethane: 638
    - Acethylene: 1450
    - DBDS: 0.0

    Other Key indicators suggest that:
    - Power factor: 1.0
    - Interfacial V: 42
    - Dielectric rigidity: 55
    - Water content: 18

    Calculated Gas ratios are:
    - Carbo

In [269]:
final_report = report + formatted_result
with open("report.txt", "w") as f:
    f.write(final_report)

# print the final_report
print(final_report)


    Transformer ID: TX-8395
    InstallationDate: 2004-10-09
    OperatingTime: 34.7907773926
    LoadCondition: Normal Load
    PerformanceMetrics: Performance stable with a load of 223.96130729828337 KW and temperature 52.76066434980563°C

    Condition Summary:
    Health Index: 60.5
    Predicted Remaining Life Expectancy: 19.0 years
    Based on the current dissolved gas analysis (DGA) and health index data, this transformer is 
    expected to last an additional 19.0 years. 

    Specific gases dissolved in the transformer insulating oil, could help identify potential faults or degradation. Current levels are:
    - Hydrogen: 9040
    - Oxygen: 1870
    - Nitrogen: 47500
    - Methane: 1790
    - CO: 183
    - CO2: 2060
    - Ethylene: 7820
    - Ethane: 638
    - Acethylene: 1450
    - DBDS: 0.0

    Other Key indicators suggest that:
    - Power factor: 1.0
    - Interfacial V: 42
    - Dielectric rigidity: 55
    - Water content: 18

    Calculated Gas ratios are:
    - Carbo

### Causal language modeling for text generation via service llm_service.py (Hugging Face transformers)

In [ ]:
with bentoml.SyncHTTPClient("http://localhost:3000") as client:
    result = client.generate(
        json_string=health_index_data.iloc[1].to_json()
    )

print(result)

In [ ]:
import requests

url = "http://localhost:3000/generate_text"
input_data = {"json_string": health_index_data.iloc[1].to_json()} 
response = requests.post(url, json=input_data)
print(response.json())


In [105]:
# convert ouput JSON to a paragraph
def convert_json_to_paragraph(json_data):
  paragraph = ""
  for key, value in json_data.items():
    paragraph += f"{key}: {value} "
  return paragraph.strip()

In [106]:
paragraph = convert_json_to_paragraph(response.json())
print(paragraph)

generated_text: 
Transformer ID: TX-1873
InstallationDate: 2002-09-27
OperatingTime: 35.26853756
LoadCondition: Normal Load
PerformanceMetrics: Performance stable with a load of 573.4742788867065 KW and temperature 92.92760626605455°C
Condition Summary:
Health Index: 63.4
Predicted Remaining Life Expectancy: 6.0 years 

Specific gases dissolved in the transformer insulating oil, could help identify potential faults or degradation. Current levels are:
- Hydrogen: 4
- Oxygen: 26100
- Nitrogen: 60600
- Methane: 1
- CO: 206
- CO2: 1440
- Ethylene: 13
- Ethane: 0
- Acethylene: 13
- DBDS: 5.0

Other Key indicators suggest that:
- Power factor: 1.32
- Interfacial V: 40
- Dielectric rigidity: 56
- Water content: 4

Calculated Gas ratios are:
- Carbon Monoxide to Hydrogen Ratio: 51.5
- Methane to Hydrogen Ratio: 0.25
- Ethylene to Hydrogen Ratio: 3.25
- Acetylene to Hydrogen Ratio: 3.25

Maintenance Information:
- Maintenance ID: MNT-26380
- Maintenance Type: Preventive Maintenance 
- Last Main